In [1]:
import pandas as pd

In [5]:
orders=pd.read_csv("orders.csv")

In [7]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [8]:
orders.tail()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian
9999,10000,457,439,21-10-2023,879.58,Classic Cafe Family Restaurant


In [9]:
users=pd.read_json("users.json")

In [10]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [11]:
users.tail()

,user_id,name,city,membership
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular
2999,3000,User_3000,Hyderabad,Gold


In [12]:
import sqlite3

In [25]:
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()

In [26]:
with open("restaurants.sql","r",encoding="utf-8") as file:
    sql_script=file.read()

cursor.executescript(sql_script)
conn.commit()

In [27]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('restaurants',)]

In [28]:
restaurants=pd.read_sql_query("SELECT * FROM restaurants",conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [29]:
restaurants.tail()

,restaurant_id,restaurant_name,cuisine,rating
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9
499,500,Restaurant_500,Chinese,3.2


In [30]:
print(orders.columns)
print(users.columns)
print(restaurants.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')
Index(['user_id', 'name', 'city', 'membership'], dtype='object')
Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')


In [33]:
orders_users=pd.merge(orders,users,how="left",on="user_id")
orders_users.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [35]:
final_dataset=pd.merge(orders_users,restaurants,how="left",on="restaurant_id")
final_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [37]:
final_dataset.to_csv("final_food_delivery_dataset.csv",index=False)
print("created successfully")

created successfully


In [38]:
final_dataset.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [41]:
gold_members=final_dataset[final_dataset["membership"]=="Gold"]

In [46]:
final_dataset["is_gold_member"] = final_dataset["membership"] == "Gold"


In [47]:
gold_members = final_dataset[final_dataset["is_gold_member"]]


In [49]:
gold_revenue_by_city = (
    final_dataset[final_dataset["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_revenue_by_city


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [50]:
avg_order_value_by_cuisine = (
    final_dataset
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

avg_order_value_by_cuisine


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [51]:
user_total_spend = (
    final_dataset
    .groupby("user_id")["total_amount"]
    .sum()
)


In [52]:
high_value_users = user_total_spend[user_total_spend > 1000]


In [53]:
num_users = high_value_users.count()
num_users


np.int64(2544)

In [54]:
rating_bins = [0, 2, 3, 4, 5]
rating_labels = ["0–2", "2–3", "3–4", "4–5"]

final_dataset["rating_range"] = pd.cut(
    final_dataset["rating"],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True
)


In [59]:
revenue_by_rating_range = (
    final_dataset
    .groupby("rating_range", observed=True)["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


In [60]:
top_range = revenue_by_rating_range.idxmax()
top_revenue = revenue_by_rating_range.max()

top_range, top_revenue


('4–5', 4157357.01)

In [63]:
total_orders = len(final_dataset)

gold_orders = len(
    final_dataset[final_dataset["membership"] == "Gold"]
)

percentage_gold_orders = round((gold_orders / total_orders) * 100)

percentage_gold_orders


50

In [64]:
restaurant_stats = final_dataset.groupby("restaurant_name_x").agg(
    total_orders=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)


In [65]:
small_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]


In [66]:
top_restaurant = small_restaurants["avg_order_value"].idxmax()
top_avg_value = small_restaurants["avg_order_value"].max()

top_restaurant, top_avg_value


('Hotel Dhaba Multicuisine', 1040.2223076923076)

In [69]:
small_restaurants.sort_values("avg_order_value", ascending=False).head(20)


,total_orders,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
Hotel Dhaba Chinese,18,973.125556
Amma Mess Pure Veg,18,965.299444
Hotel Biryani Pure Veg,13,964.577692
Annapurna Curry House Multicuisine,17,954.512353


In [70]:
restaurants_of_interest = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

subset = final_dataset[
    final_dataset["restaurant_name_x"].isin(restaurants_of_interest)
]


In [71]:
avg_order_value_subset = subset.groupby("restaurant_name_x")["total_amount"].mean()
avg_order_value_subset


restaurant_name_x
Grand Cafe Punjabi               765.409063
Grand Restaurant South Indian    842.567586
Ruchi Foods Chinese              686.603158
Ruchi Mess Multicuisine          851.226250
Name: total_amount, dtype: float64

In [72]:
top_restaurant = avg_order_value_subset.idxmax()
top_avg_value = avg_order_value_subset.max()

top_restaurant, top_avg_value


('Ruchi Mess Multicuisine', 851.22625)

In [74]:
top_combo = combo_revenue.idxmax()
top_revenue = combo_revenue.max()

top_combo, top_revenue


(('Swagath Cafe Multicuisine', 'Chinese'), 45821.74)

In [75]:
final_dataset["membership_cuisine"] = final_dataset["membership"] + " + " + final_dataset["cuisine"]


In [76]:
combinations_of_interest = [
    "Gold + Indian",
    "Gold + Italian",
    "Regular + Indian",
    "Regular + Chinese"
]

subset = final_dataset[final_dataset["membership_cuisine"].isin(combinations_of_interest)]


In [77]:
combo_revenue = subset.groupby("membership_cuisine")["total_amount"].sum()
combo_revenue


membership_cuisine
Gold + Indian         979312.31
Gold + Italian       1005779.05
Regular + Chinese     952790.91
Regular + Indian      992100.27
Name: total_amount, dtype: float64

In [78]:
top_combo = combo_revenue.idxmax()
top_revenue = combo_revenue.max()

top_combo, top_revenue


('Gold + Italian', 1005779.05)

In [80]:
final_dataset["order_date"] = pd.to_datetime(final_dataset["order_date"], dayfirst=True)



In [81]:
final_dataset["order_date"] = pd.to_datetime(final_dataset["order_date"], format="%d-%m-%Y")


In [82]:
final_dataset["quarter"] = final_dataset["order_date"].dt.quarter

revenue_by_quarter = final_dataset.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)

top_quarter_num = revenue_by_quarter.idxmax()
top_revenue = revenue_by_quarter.max()

quarter_mapping = {1: "Q1 (Jan–Mar)", 2: "Q2 (Apr–Jun)", 3: "Q3 (Jul–Sep)", 4: "Q4 (Oct–Dec)"}
top_quarter = quarter_mapping[top_quarter_num]

top_quarter, top_revenue


('Q3 (Jul–Sep)', 2037385.1)

In [86]:
gold_orders_count = final_dataset[final_dataset["membership"] == "Gold"].shape[0]
gold_orders_count



4987

In [87]:
hyderabad_revenue = final_dataset[final_dataset["city"] == "Hyderabad"]["total_amount"].sum()
hyderabad_revenue_rounded = round(hyderabad_revenue)
hyderabad_revenue_rounded


1889367

In [88]:
revenue_by_city = gold_members.groupby("city")["total_amount"].sum()
top_city = revenue_by_city.idxmax()


In [89]:
orders_in_top_city = gold_members[gold_members["city"] == top_city].shape[0]
orders_in_top_city


1337

In [90]:
print(f"Top revenue city among Gold members: {top_city}")
print(f"Number of orders in this city by Gold members: {orders_in_top_city}")


Top revenue city among Gold members: Chennai
Number of orders in this city by Gold members: 1337


In [91]:
high_rating_orders = final_dataset[final_dataset["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [92]:
distinct_users = final_dataset["user_id"].nunique()
distinct_users


2883

In [93]:
avg_aov_gold = round(
    final_dataset[final_dataset["membership"] == "Gold"]["total_amount"].mean(), 2
)
avg_aov_gold


np.float64(797.15)